# Se på ord i data

In [ ]:
from ordbilder.annotations import get_annotation_information
from samisk_ocr.clean_text_data import clean
from pathlib import Path

train_path = Path("../data/transkribus_exports/train_data/train")
s_30_path = Path("../data/transkribus_exports/train_data/side_30")
test_path = Path("../data/transkribus_exports/test_data/2997983/Testsett_Samisk_OCR")


# Lag txt-mappe hvis ikke finnes
for path in (train_path, s_30_path, test_path):
    was_unclean = 0
    num_pages = 0
    for alto_dir in path.glob("**/alto"):
        files = list(alto_dir.glob("*.xml"))
        text_dir = alto_dir.parent / "txt"
        text_dir.mkdir(exist_ok=True)
        for xml_file in files:
            num_pages += 1
            page_file = text_dir / f"{xml_file.stem}.txt"
            annotations = get_annotation_information(xml_path=xml_file)
            page_text_pre = "\n".join([e["word"] for e in annotations])
            page_text = clean(page_text_pre)
            if page_text != page_text_pre:
                was_unclean += 1
            with page_file.open("w+") as f:
                f.write(page_text)
    print(
        f"I {path.name} var det {was_unclean} sider (av {num_pages} totalt) med uønskede tegn (non-breaking space og em-dash)"
    )

In [ ]:
from nb_tokenizer import tokenize
from collections import Counter


def get_tokens(directory: Path) -> Counter[str]:
    tokens = []
    for text_dir in directory.glob("**/txt"):
        text_files = text_dir.glob("*.txt")
        texts = " ".join([e.read_text() for e in text_files])
        tokens += tokenize(texts)
    return Counter(tokens)


train_tokens = get_tokens(train_path)
test_tokens = get_tokens(test_path)
s_30_tokens = get_tokens(s_30_path)

In [ ]:
print(f"""Unike tokens:
    Train:      {'{:_}'.format(len(train_tokens))}
    Side 30:    {'{:_}'.format(len(s_30_tokens))}
    Test:       {'{:_}'.format(len(test_tokens))}
""")

In [ ]:
print(f"""Totalt antall tokens:
    Train:      {'{:_}'.format(sum(train_tokens.values()))}
    Side 30:    {'{:_}'.format(sum(s_30_tokens.values()))}
    Test:       {'{:_}'.format(sum(test_tokens.values()))}
""")

In [ ]:
from typing import Iterable, Any


def percent_overlap(container1: Iterable[Any], container2: Iterable[Any]) -> float:
    intersection_len = len(set(container1).intersection(set(container2)))
    intersection_part = intersection_len / len(container1)
    return round(intersection_part * 100, 2)

In [ ]:
print(f"""Tokenoverlapp
      
    Train og test:      
        {percent_overlap(test_tokens, train_tokens)}% av tokensa i test finnes i train
        {percent_overlap(train_tokens, test_tokens)}% av tokensa i train finnes i test

    Train og side 30:   
        {percent_overlap(train_tokens, s_30_tokens)}% av tokensa i train finnes i side 30
        {percent_overlap(s_30_tokens, train_tokens)}% av tokensa i side 30 finnes i train 

    Test og side 30:   
        {percent_overlap(test_tokens, s_30_tokens)}% av tokensa i test finnes i side 30
        {percent_overlap(s_30_tokens, test_tokens)}% av tokensa i side 30 finnes i test       
      
""")

## Forskjeller i tokens mellom settene

In [ ]:
print(
    f"Antall tokens som bare fins i side 30:  {len(set(s_30_tokens) - (set(train_tokens).union(set(test_tokens))))}"
)
print(
    f"Antall tokens som bare fins i train:    {len(set(train_tokens) - set(s_30_tokens).union(set(test_tokens)))}"
)
print(
    f"Antall tokens som bare fins i test:     {len(set(test_tokens) - set(s_30_tokens).union(set(train_tokens)))}"
)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud


def get_text(directory: Path) -> str:
    text = ""
    for text_dir in directory.glob("**/txt"):
        text_files = text_dir.glob("*.txt")
        texts = " ".join([e.read_text() for e in text_files])
        text += texts
    return text


train_text = get_text(train_path)
train_cloud = WordCloud().generate(train_text)
plt.figure(figsize=(60, 20))
plt.imshow(train_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
test_text = get_text(test_path)
test_cloud = WordCloud().generate(test_text)
plt.figure(figsize=(60, 20))
plt.imshow(test_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
s_30_text = get_text(s_30_path)
s_30_cloud = WordCloud().generate(s_30_text)
plt.figure(figsize=(60, 20))
plt.imshow(s_30_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Se på bokstaver og tegn i data

In [ ]:
train_chars = Counter(train_text)
s_30_chars = Counter(s_30_text)
test_chars = Counter(test_text)

print(f"""Unike tegn:
    Train:      {'{:_}'.format(len(train_chars))}
    Side 30:    {'{:_}'.format(len(s_30_chars))}
    Test:       {'{:_}'.format(len(test_chars))}
""")

print(f"""Totalt antall tegn:
    Train:      {'{:_}'.format(sum(train_chars.values()))}
    Side 30:    {'{:_}'.format(sum(s_30_chars.values()))}
    Test:       {'{:_}'.format(sum(test_chars.values()))}
""")

print(f"""Tegnoverlapp
      
    Train og test:      
        {percent_overlap(test_chars, train_chars)}% av tegna i test finnes i train
        {percent_overlap(train_chars, test_chars)}% av tegna i train finnes i test

    Train og side 30:   
        {percent_overlap(train_chars, s_30_chars)}% av tegna i train finnes i side 30
        {percent_overlap(s_30_chars, train_chars)}% av tegna i side 30 finnes i train 

    Test og side 30:   
        {percent_overlap(test_chars, s_30_chars)}% av tegna i test finnes i side 30
        {percent_overlap(s_30_chars, test_chars)}% av tegna i side 30 finnes i test       
      
""")

## Forskjeller i tegn mellom settene

In [ ]:
print(
    f"Bare i side 30: {list(set(s_30_chars) - (set(train_chars).union(set(test_chars))))}"
)

In [ ]:
print(f"Bare i train {list(set(train_chars) - set(s_30_chars).union(set(test_chars)))}")